# INFO279 - Tarea 1: Tratamiento Automático del Lenguaje


## Desafíos:

1. **Clasificación de noticias**: Construir un modelo capaz de clasificar noticias según su categoría temática.
2. **Geolocalización de eventos**: Extraer información de noticias, incluyendo el evento principal, dirección y coordenadas geográficas (latitud, longitud).

**Categorías**: sociedad, salud, politica, medioambiente, internacional, entretenimiento, economia, deportes, cultura, cienciatecnologia



In [1]:

# Importar librerías necesarias
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
import spacy
from geopy.geocoders import Nominatim


In [6]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Embedding, Dropout, LSTM, BatchNormalization
from tensorflow.keras.preprocessing.sequence import pad_sequences
from transformers import BertTokenizer
import numpy as np
import spacy
import re

# Cargar modelo de SpaCy para procesamiento de texto en español
nlp = spacy.load('es_core_news_sm')

def preprocess_text(text):
    # Preprocesar texto: eliminar caracteres especiales y convertir a minúsculas
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Eliminar caracteres especiales
    return text.lower()  # Convertir todo a minúsculas

print("Cargando el dataset")

# Cargar dataset de ejemplo para clasificación
df = pd.read_csv('data/train_data.csv')  # Cambiar por la ruta real del dataset
print("Dataset cargado")

# Combinar las columnas 'title' y 'text' en una sola columna de contenido (si deseas)
df['content'] = df['title'] + " " + df['text']
print("Columnas combinadas")

# Preprocesar el dataset (título y texto)
df['content'] = df['content'].apply(preprocess_text)
print("Texto preprocesado")

# Codificación de etiquetas
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['clase'])
print("Etiquetas codificadas")



tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')  # Usar BERT
print("Tokenizador BERT cargado")


def encode_text(texts):
    return tokenizer(texts.tolist(), padding=True, truncation=True, return_tensors="np", max_length=128)

print("Codificando los textos")

# Codificar los textos de entrenamiento
encodings = encode_text(df['content'])

# Obtener la longitud máxima de las secuencias para el padding
max_length = 128  # Reducido a 128 para optimizar el tiempo de entrenamiento
print("Longitud máxima de secuencias:", max_length)

# Padding de las secuencias
X_pad = pad_sequences(encodings['input_ids'], maxlen=max_length, padding='post')
y = df['label'].values
print("Secuencias de entrada preparadas")

# Definir el modelo de red neuronal convolucional 
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.vocab) + 1, output_dim=100))  # Usando embedding preentrenado si lo tienes
print("Capa de Embedding añadida")
model.add(Conv1D(filters=64, kernel_size=5, activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.5))  # Regularización
model.add(LSTM(units=32, return_sequences=True))  
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(len(label_encoder.classes_), activation='softmax'))  
print("Modelo definido")

# Compilar el modelo
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
print("Modelo compilado")

# Entrenar el modelo con todo el dataset durante 10 épocas completas
model.fit(X_pad, y, epochs=5, batch_size=16, validation_split=0.2)  # Entrenará durante 10 épocas completas
print("Modelo entrenado durante 10 épocas")


model.save("modelo_clasificador_noticias.keras")
print("Modelo guardado")

# Cargar el modelo entrenado
loaded_model = load_model("modelo_clasificador_noticias.keras")
print("Modelo cargado")

# Compilar el modelo cargado (para evaluación)
loaded_model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print("Modelo compilado para evaluación")

# Predicción y evaluación en el mismo dataset
y_pred = np.argmax(loaded_model.predict(X_pad), axis=-1)

# Reporte de evaluación
from sklearn.metrics import accuracy_score, classification_report

print("Predicciones realizadas")
print("Accuracy:", accuracy_score(y, y_pred))
print(classification_report(y, y_pred, target_names=label_encoder.classes_))





Cargando el dataset
Dataset cargado
Columnas combinadas
Texto preprocesado
Etiquetas codificadas
Tokenizador BERT cargado
Codificando los textos
Longitud máxima de secuencias: 128
Secuencias de entrada preparadas
Capa de Embedding añadida
Modelo definido
Modelo compilado
Epoch 1/5
1100/1100 ━━━━━━━━━━━━━━━━━━━━ 117s 103ms/step - accuracy: 0.2525 - loss: 1.9836 - val_accuracy: 0.0730 - val_loss: 8.6766
Epoch 2/5
1100/1100 ━━━━━━━━━━━━━━━━━━━━ 115s 104ms/step - accuracy: 0.5549 - loss: 1.2947 - val_accuracy: 0.0707 - val_loss: 12.3208
Epoch 3/5
1100/1100 ━━━━━━━━━━━━━━━━━━━━ 115s 104ms/step - accuracy: 0.6838 - loss: 0.9489 - val_accuracy: 0.0743 - val_loss: 13.8270
Epoch 4/5
1100/1100 ━━━━━━━━━━━━━━━━━━━━ 113s 103ms/step - accuracy: 0.7559 - loss: 0.7290 - val_accuracy: 0.0614 - val_loss: 15.6756
Epoch 5/5
1100/1100 ━━━━━━━━━━━━━━━━━━━━ 114s 104ms/step - accuracy: 0.8103 - loss: 0.5694 - val_accuracy: 0.0593 - val_loss: 18.1909
Modelo entrenado durante 10 épocas
Modelo guardado
Modelo c

C:\Users\crist\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\crist\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\crist\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} i

In [5]:
import pandas as pd
import numpy as np
import tensorflow as tf
from transformers import TFBertModel, BertTokenizer
import spacy
from geopy.geocoders import Nominatim
from sklearn.preprocessing import LabelEncoder


print("Cargando spaCy...")
nlp = spacy.load('es_core_news_sm')
print("spaCy cargado.")

# Inicializando geolocalizador
print("Geolocalizador inicializado.")
geolocator = Nominatim(user_agent="geoapiExercises")

# Cargando el modelo de clasificación
print("Cargando el modelo de clasificación...")
modelo_clasificador = tf.keras.models.load_model("modelo_clasificador_noticias.keras")
print("Modelo cargado.")

# Cargando el dataset
print("Cargando el dataset...")
df = pd.read_csv('data/dataset_agosto2024.csv')

# Seleccionar solo 100 noticias del dataset
df = df.sample(n=100, random_state=42)

# Preprocesamiento del texto (combinando título y texto)
print("Preprocesando el texto (combinando título y texto)...")
df['texto_combinado'] = df['title'] + ' ' + df['text']
df['texto_combinado'] = df['texto_combinado'].apply(lambda x: str(x).lower())
print("Texto combinado y preprocesado.")

# Codificación de texto
print("Texto codificado...")
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
inputs = tokenizer(df['texto_combinado'].tolist(), padding=True, truncation=True, return_tensors="tf", max_length=512)


print("Obteniendo embeddings de BERT...")
bert_model = TFBertModel.from_pretrained('bert-base-multilingual-cased')
bert_embeddings = bert_model(inputs['input_ids'])[0]  


cls_embeddings = bert_embeddings[:, 0, :]  

# Ajustar la forma para que coincida con la entrada esperada por el modelo
print(f"Forma antes de la capa Dense: {cls_embeddings.shape}")
cls_embeddings_resized = tf.keras.layers.Dense(128, activation='relu')(cls_embeddings)

# Verificando la forma de los embeddings ajustados
print(f"Forma de los embeddings ajustados: {cls_embeddings_resized.shape}")

# Predecir usando el modelo cargado
print("Prediciendo usando el modelo cargado...")
predicciones = modelo_clasificador.predict(cls_embeddings_resized)

# Decodificando las predicciones
label_encoder = LabelEncoder()
label_encoder.fit(df['media_outlet'].unique()) 

predicciones_codificadas = np.argmax(predicciones, axis=1)
predicciones_decodificadas = label_encoder.inverse_transform(predicciones_codificadas)

# Añadir predicciones al DataFrame
df['category'] = predicciones_decodificadas

# Función para extraer eventos de las noticias
def extraer_evento(texto):

    return '.'.join(texto.split('.')[:1])

df['event'] = df['texto_combinado'].apply(extraer_evento)

# Función para extraer ubicaciones del texto usando spaCy
def extraer_ubicaciones(texto):
    doc = nlp(texto)
    ubicaciones = [ent.text for ent in doc.ents if ent.label_ == 'LOC']  # Detectamos entidades etiquetadas como 'LOC' (lugares)
    return ubicaciones[0] if ubicaciones else None

# Aplicar la detección de ubicaciones al texto combinado
print("Extrayendo ubicaciones de los textos...")
df['address'] = df['texto_combinado'].apply(extraer_ubicaciones)

# Geolocalización basada en las ubicaciones detectadas
df['latitud'] = np.nan
df['longitud'] = np.nan

for i, row in df.iterrows():
    address = row['address']
    if address:  # Si se detectó al menos una ubicación
        print(f"Geolocalizando '{address}' para la noticia {i+1}...")
        try:
            location = geolocator.geocode(address)
            if location:
                df.at[i, 'latitud'] = location.latitude
                df.at[i, 'longitud'] = location.longitude
        except Exception as e:
            print(f"Error al geolocalizar '{address}': {e}")

print("Geolocalización completada.")


df_resultado = df[['id_news','event', 'category', 'address', 'latitud', 'longitud']]
df_resultado.to_csv('resultados_geolocalizacion.csv', index=False)

print("Proceso completado y resultados guardados.")


Cargando spaCy...
spaCy cargado.
Geolocalizador inicializado.
Cargando el modelo de clasificación...
Modelo cargado.
Cargando el dataset...
Preprocesando el texto (combinando título y texto)...
Texto combinado y preprocesado.
Texto codificado...
Obteniendo embeddings de BERT...


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

Forma antes de la capa Dense: (100, 768)
Forma de los embeddings ajustados: (100, 128)
Prediciendo usando el modelo cargado...
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
Extrayendo ubicaciones de los textos...
Geolocalizando 'chile' para la noticia 10405...
Geolocalizando 'coquimbo' para la noticia 20027...
Geolocalizando '¿' para la noticia 2044...
Geolocalizando 'temuco' para la noticia 11509...
Geolocalizando 'chile' para la noticia 8399...
Geolocalizando 'calle albany' para la noticia 15575...
Geolocalizando 'rn ximena ossandón' para la noticia 11332...
Geolocalizando 'talca' para la noticia 9093...
Geolocalizando 'venezuela' para la noticia 14497...
Geolocalizando 'lyon de providencia' para la noticia 4721...
Geolocalizando 'coquimbo' para la noticia 10651...
Geolocalizando 'betis' para la noticia 3474...
Geolocalizando 'carolina' para la noticia 5355...
Geolocalizando 'corea del sur' para la noticia 14627...
Geolocalizando 'chile' para la noticia 7871...
Geolocalizando 'chile' para la